In order to find these quantities, we measured the temperature and humidity daily. Barometric pressure is the same through large areas, so this can be found on weather records.

Temperature fluctuated significantly during lab meetings. I recorded $65.5^\circ F \rightarrow 66.2^\circ F$ on January 27th, so I'll use a higher value for temperature ($\sigma_F = 1^\circ C$) uncertainty than I could. A cleverer scientist would have temperature measurements automatically recorded into their data logs. I would also like to note that writing the temperature in fahrenheit is actually preferable because, once converted there is less uncertainty in the temperature. In fact, an error of $1^\circ F$ corresponds to an error of $0.56^\circ C$, nearly doubling our precision. Of course this also makes the numbers look like they were taken from a more accurate machine that they were. The hundredth's place is misleading, but it does contain some information. 

$\sigma_T = 0.56^\circ C$

|  Date  | T(C)  | Humidity |
|--------|-------|----------|
|  1/25  | 19.2  | 20%      |
|  1/27  | 19.61 | 20%      |
|  2/01  | 21.61 | 20%      |


I should consider writing a script to write the temperature and humidity into all of my data sheets, based off this chart.

The areal density of a a sample is defined as

$$\tau \equiv \rho L$$
$$\tau \equiv (\rho_d + \rho_v)L$$
$$\tau \equiv (\frac{P_d}{R_dT} + \frac{P_v}{R_vT})L$$
$R_d = 287.058 J/Kg \cdot K$
$R_d = 461.495 J/Kg \cdot K$

$P$, the total barometric pressure is something that we record. The dry and vapor pressures must add to this value. The saturation vapor pressure times the humidity gives the vapor pressure in air.

$$P_v = \phi \times P_{sat}$$
Unfortunately the saturation pressure is determined by
$$P_{sat} = 610.78 \times 10^{\frac{7.5T-2048.625}{T-35.85}}$$

In [39]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.misc import derivative

TC = [19.2, 19.61, 21.61]
TK = [t+273.15 for t in TC]
# print(TK)

P_sat = lambda T: 610.78*10**((7.5*T-2048.625)/(T-35.85))
# print([P_sat(t) for t in TK])

TK = np.arange(292, 295, 0.01)
P_s = np.array([P_sat(t) for t in TK])
# plt.plot(TK, P_s)

def dP_vdT(t):
    return np.gradient(P_s, 0.1)[int((t-292)/0.01)]

def σ_P_sat(T, σ_T):
    return dP_vdT(T)*σ_T

TK = [t+273.15 for t in TC]

for T in TK:
    print(T, P_sat(T), σ_P_sat(T, 0.56))

292.34999999999997 2224.786376254321 7.756033133033816
292.76 2282.2412488270243 7.930972042694157
294.76 2581.5142921761017 8.832988385347017


In [50]:
Φ = 0.20
σ_Φ = 0.01
P_v = []
σ_P_v = []
for T in TK:
    P_v.append(P_sat(T)*Φ)
    σ_P_v.append(P_sat(T)*Φ*σ_Φ)
    
    print("P_v   = ", P_sat(T)*Φ, "N/m^2")
    print("σ_P_v = ", np.sqrt(P_sat(T)**2*σ_Φ**2 + σ_P_sat(T, 0.56)**2*Φ**2), "N/m^2\n")

P_v = np.array(P_v)
σ_P_v = np.array(σ_P_v)

P_v   =  444.95727525086426 N/m^2
σ_P_v =  22.301876243830073 N/m^2

P_v   =  456.44824976540485 N/m^2
σ_P_v =  22.877467615241713 N/m^2

P_v   =  516.3028584352204 N/m^2
σ_P_v =  25.87551876626969 N/m^2



In [68]:
weather = np.array([
    [29.31, 29.26, 29.19],
    [29.43, 29.07, 28.67],
    [29.45, 29.08, 28.82]
])

P = weather[:, 1]
σ_P = np.sqrt((weather[:,0] - weather[:,1])**2+(weather[:,1]-weather[:,2])**2)
P *= 101_325 / 29.9212
σ_P *= 101_325 / 29.9212
print(f'{P=}')
print(f'{σ_P=}')

P_d = P - P_v
σ_P_d = np.sqrt(σ_P**2 + σ_P_v**2)

P=array([99085.91567183, 98442.500635  , 98476.36458431])
σ_P=array([ 291.30870676, 1822.37137089, 1531.38455103])


In [69]:
def τ(P_torr, T, Φ):
    '''
    @param T: temperature in °C
    @param P_torr: pressure in torr
    @param Φ: humidity in %
    @returns a function of linear distance in meters which represents the areal density.
    '''
    T = T + 273.15
    P = P_torr * 133.3224
    P_v = P_sat(T)*Φ
    P_d = P - P_v
    R_d = 287.058
    R_v = 461.495
    return lambda L : (L/T)*(P_d/R_d + P_v/R_v)

def σ_τ(T, P_torr, Φ, σ_T, σ_P_torr, σ_Φ):
    '''
    @param T: temperature in °C 
    @param P_torr: pressure in torr
    @param Φ: humidity in %
    @param σ_T: temperature uncertainty in °C
    @param σ_P_torr: pressure uncertainty in torr
    @param σ_Φ: humidity uncertainty in %
    @returns a function of linear distance in meters which represents the uncertainty in areal density.

    @todo finish this implementation
    '''
    arealDensity = τ(T, P_torr, Φ)
    T = T + 273.15
    return lambda L, σ_L: np.sqrt(
        arealDensity(L)**2 * ( σ_T**2/T**2 + σ_L**2/L**2 ) + L**2/T**2
    )